In [ ]:
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../')

from train import run, set_arguments

parser = set_arguments()
args = parser.parse_args([
    '--data_path', '../../data/formatted/formatted_train_cat1_text_1M_converted.csv',
    '--label_column', 'cat1',
    '--text_column', 'text',
    '--max_rows', '1000000',
    '--encoding', 'utf-8',
    '--alphabet', """ضصثقفغعهخحجچشسیبلاتنمکگظطزرذدپوآ۱۲۳۴۵۶۷۸۹۰0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}""",
    '--number_of_characters', '84',
    '--extra_characters', '',
    '--epochs', '2',
    '--config', '../config.json',
    '--number_of_classes', '10',
])


In [ ]:
from time import time
start_time = time()
run(args)
with open("clcnn_time_train.txt", "w") as file:
    file.write(str(time() - start_time))

In [ ]:
#Testing the thingee
import pandas as pd
test_data = pd.read_csv("../../data/formatted/formatted_test_cat1_text_1M_converted.csv")
model = './models/char_cnn_epoch_None_1_0.01_loss_0.9538_acc_0.6496.pth' # model name here!!

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from time import time
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../')

from predict import predict, set_arguments

sys.path.append('../../')
from helper import unconvert_label_cat1

initial_args = [
    '--alphabet', """ضصثقفغعهخحجچشسیبلاتنمکگظطزرذدپوآ۱۲۳۴۵۶۷۸۹۰0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}""",
    '--number_of_characters', '84',
    '--extra_characters', '',
#     '--config', '../config.json',
]
parser = set_arguments()
total = 0
y_real = []
y_pred = []
total_time = 0
for index, row in test_data.iterrows():
#     print(index)
    text = row["text"]
    pred = row["cat1"]
    args = parser.parse_args(initial_args + ['--model', model, '--text', text])
    start_time = time()
    res = predict(args)
    infer = unconvert_label_cat1(np.argmax(list(res[0])))
    total_time += time() - start_time
#     print(list(res[0]))
    y_real.append(unconvert_label_cat1(pred))
    y_pred.append(infer)
    total += 1
    
    if index > 0 and index % 1000 == 0:
        print("----------------------------------")
        print(classification_report(y_real, y_pred))
        print("Average Infer Time: ", total_time / total)

In [ ]:
results = pd.DataFrame([[y_real[i], y_pred[i]] for i in range(len(y_pred))], columns=["y_real", "y_pred"])
results.to_csv(open("results_clcnn_1M.csv", "w"), index=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_real, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
from helper import get_labels, plot_confusion_matrix


np.set_printoptions(precision=2)
unique_labels(y_real, y_pred)
plot_confusion_matrix(y_real, y_pred, classes=get_labels(), normalize=True)
# plt.show()